### This program solves the stochastic growth model labor leisure choice with linear quadratic dynamic programming.

#### Translated from Eva Carceles-Poveda's (2003) MATLAB codes

In [1]:
# Importing packages
import numpy as np

# needed for compact printing of numpy arrays
# use precision to set the number of decimal digits to display
# use suppress=True to show values in full decimals instead of using scientific notation
np.set_printoptions(suppress=True,precision=4,linewidth=np.inf)

#### The riccati1 (Ricatti difference equation) function is copied below

In [2]:
def riccati1(U,DJ,DH,S,C,B,Sigma,beta):
    """ Syntax: [P1,J,d] = riccati(U,DJ,DH,S,C,B,Sigma,beta)
    
    This function solves for the value and policy functions of a linear quadratic problem 
    by iterating on the Ricatti difference equation. The inputs are the return function at 
    steady state (U), the Jacobian and Hessian matrices (DJ and DH), the vector of states 
    (S=[z; s]), the vector of controls (C=[x]), the matrix B satisfying [1;z';s']=B[1;z;s;x] 
    the variance covariance matrix of [1 z' s'] (Sigma) and the discount factor beta.
    
    Translated from Eva Carceles-Poveda 2003 MATLAB code
    """

    tolv = 1e-07

    ns1, ns2 = S.shape
    if ns2 > ns1:
        S = S.T

    ns = max(S.shape)

    nc1, nc2 = C.shape
    if nc2 > nc1:
        C = C.T
    
    nc = max(C.shape)

    WW = np.concatenate((S.T,C.T),axis=1).T
    Q11 = U - WW.T@DJ + 0.5*WW.T@DH@WW
    Q12 = 0.5*(DJ-DH@WW)
    Q22 = 0.5*DH
        
    QQ = np.concatenate((np.concatenate((Q11, Q12.T),axis=1),np.concatenate((Q12,Q22),axis=1)))

    nq=ns+nc+1

    # Partition Q to separate states and controls 
    Qff = QQ[0:ns+1,0:ns+1]
    Qfx = QQ[ns+1:nq,0:ns+1]
    Qxx = QQ[nq-nc:nq,nq-nc:nq]
    
    # Initialize matrices
    P0 = -0.1*np.eye((ns+1))
    P1 = np.ones((ns+1,ns+1))
    
    # Iterate on Bellman's equation until convergence
    while np.linalg.norm(np.abs(P1-P0)) > tolv:
        P1 = P0.copy()
        M = B.T@P0@B;
        Mff = M[0:ns+1,0:ns+1]
        Mfx = M[ns+1:nq,0:ns+1]
        Mxx = M[nq-nc:nq,nq-nc:nq]

        P0=Qff+beta*Mff-(Qfx+beta*Mfx).T@np.linalg.inv(Qxx+beta*Mxx)@(Qfx+beta*Mfx)   

      

    J = -np.linalg.inv(Qxx + beta*Mxx)@(Qfx + beta*Mfx)

    d = beta/(1-beta)*np.trace(P0@Sigma)

    return P1,J,d

#### The Hodrick-Prescott filter function is copied below

In [3]:
def hp1(y,w):

    """ Syntax: yhp, ytr = hp1(y, w)
    
    This function computes the filtered series of y, using
    a smoothing parameter w. 
    
    The code is from I. Izvorski.
    """

    t, s = y.shape
    
    if t < s:
        y = y.T

    a = 6*w + 1
    b = -4*w
    c = w
    d = np.array([[c,b,a]])
    d = np.ones((t,1))*d
    m = np.diag(d[:,2])+np.diag(d[0:-1,1],1)+np.diag(d[0:-1,1],-1)
    m = m+np.diag(d[0:-2,0],2)+np.diag(d[0:-2,0],-2)

    m[0,0] = 1+w;       m[0,1] = -2*w;
    m[1,0] = -2*w;      m[1,1] = 5*w+1;
    m[-2,-2] = 5*w+1;   m[-2,-1] = -2*w;
    m[-1,-2] = -2*w;    m[-1,-1] = 1+w;
    
    ytr = np.matmul(np.linalg.inv(m),y)
    yhp = y-ytr

    return yhp, ytr

In [4]:
# Model Parameters
beta    = 0.99     # Discount factor
delta   = 0.025    # Depreciations
alpha   = 0.36     # Capital's share
rho     = 0.95     # Autocorrelation of shock
gamma   = 1        # One plus the quarterly growth rate of technology
sigmae  = 0.00712

In [5]:
# Steady state
zbar    =  1
hbar    =  0.3
kbar    =  hbar * (((gamma/beta - (1- delta )) / alpha )**(1/(alpha - 1)))
ibar    =  (gamma - 1 + delta) * kbar
ybar    =  (kbar**alpha)*(hbar**(1-alpha))
cbar    =  ybar - ibar
prodbar =  ybar / hbar
Rbar    =  alpha*(kbar**(alpha-1))*(hbar**(1-alpha))
wbar    =  (1-alpha)*(kbar**alpha)*(hbar**(-alpha))
a       =  (1-hbar)*wbar/cbar
print('The steady state values of z, y, i, c, k and h are:',np.array([zbar, ybar, ibar, cbar, kbar, hbar]))


The steady state values of z, y, i, c, k and h are: [ 1.      1.1112  0.2849  0.8263 11.3968  0.3   ]


In [6]:
# Obtain a quadratic approximation of the return function
Ubar    =  np.log(cbar) + a*np.log(1 - hbar)

# Construct the quadratic expansion of the utility function
Uz  = (kbar**alpha)*(hbar**(1-alpha))/cbar
Uk  = alpha*zbar*(kbar**(alpha-1))*(hbar**(1-alpha))/cbar
Ui  = -1/cbar
Uh  = (1-alpha)*zbar*(kbar**alpha)*(hbar**(-alpha))/cbar -a/(1-hbar)
DJ  = np.array([[Uz],[Uk],[Ui],[Uh]])

c2  = cbar**2
Ukk = ((alpha-1)*alpha*zbar*(kbar**(alpha -2))*(hbar**(1-alpha))*cbar \
    -(alpha*zbar*(kbar**(alpha-1))*(hbar**(1-alpha))) \
    *(alpha*zbar*(kbar**(alpha-1))*(hbar**(1-alpha))) )/c2

Ukz = ((alpha*(kbar**(alpha-1))*(hbar**(1-alpha))*cbar \
    -(alpha*zbar*(kbar**(alpha-1))*(hbar**(1-alpha))) \
    *(kbar**alpha)*(hbar**(1-alpha))) )/c2

Uki = alpha*zbar*(kbar**(alpha-1))*(hbar**(1-alpha))/c2

Ukh = (((1-alpha)*alpha*zbar*(kbar**(alpha-1))*(hbar**(-alpha)))*cbar \
    -(alpha*zbar*(kbar**(alpha-1))*(hbar**(1-alpha))) \
    *((1-alpha)*zbar*(kbar**alpha)*(hbar**(-alpha))))/c2

Uzz = -((kbar**(alpha))*(hbar**(1-alpha)) \
    *(kbar**(alpha))*(hbar**(1-alpha)))/c2

Uzi = (kbar**(alpha))*(hbar**(1-alpha))/c2

Uzh = ((1-alpha)*(kbar**(alpha))*(hbar**(-alpha))*cbar \
    -(kbar**(alpha))*(hbar**(1-alpha)) \
    *((1-alpha)*zbar*(kbar**(alpha))*(hbar**(-alpha))))/c2

Uii = -1/c2
Uih = ((1-alpha)*zbar*(kbar**(alpha))*(hbar**(-alpha)))/c2

Uhh = ((-alpha*(1-alpha)*zbar*(kbar**(alpha))*(hbar**(-alpha-1)))*cbar \
    - ((1-alpha)*zbar*(kbar**(alpha))*(hbar**(-alpha))) \
    *((1-alpha)*zbar*(kbar**(alpha))*(hbar**(-alpha))))/c2 -a/((1-hbar)**2);

DH = np.array([[Uzz, Ukz, Uzi, Uzh], [Ukz, Ukk, Uki, Ukh], [Uzi, Uki, Uii, Uih], [Uzh, Ukh, Uih, Uhh]])

S = np.array([[zbar], [kbar]])
C = np.array([[ibar], [hbar]])

B = np.array([[1, 0, 0, 0, 0], [1-rho, rho, 0, 0, 0], [0 , 0, 1-delta, 1, 0]])

Sigma = np.array([[0, 0, 0], [0, sigmae**2, 0], [0, 0, 0]])

In [7]:
# Calculate the value function
P,J,d = riccati1(Ubar,DJ,DH,S,C,B,Sigma,beta)

In [8]:
print(' The optimal value function is [1 z s]P0[1; z; s]+d, where P and d are given by:')
print(P)
print(round(d,4))

print(' The policy function is x=J[1; z; s] where J is:')
print(J)

 The optimal value function is [1 z s]P0[1; z; s]+d, where P and d are given by:
[[-138.1161   15.6227    1.186 ]
 [  15.6227   -1.9081   -0.2118]
 [   1.186    -0.2118   -0.0319]]
-0.0096
 The policy function is x=J[1; z; s] where J is:
[[-0.7867  1.3249 -0.0222]
 [ 0.1494  0.2289 -0.0069]]


In [9]:
# simulate the artificial economy
T = 115
N = 100
ss_mat = np.zeros((N,6))
cc_mat = np.zeros((N,6))
rng = np.random.Generator(np.random.MT19937())

for j in np.arange(0,N):
    r = rng.standard_normal((T+1,1))
    z = np.ones((T+1,1))
    z[0] = 1
    k = np.zeros((T+1,1))
    k[0] = kbar
    i = np.zeros((T,1))
    c = np.zeros((T,1))
    y = np.zeros((T,1))
    h = np.zeros((T,1))
    prod = np.zeros((T,1))

    for t in np.arange(0,T):
        i[t] = J[0,:]@np.array([[1],z[t],k[t]])
        h[t] = J[1,:]@np.array([[1],z[t],k[t]])
        y[t] = z[t]*k[t]**alpha*h[t]**(1-alpha)

        c[t] = y[t]-i[t]
        k[t+1] = (1-delta)*k[t] + i[t]
        z[t+1] = 1-rho+rho*z[t]+sigmae*r[t]
        prod[t] = y[t]/h[t]
        

    z = z[0:T]
    k = np.log(k[0:T])
    y = np.log(y[0:T])
    c = np.log(c)
    i = np.log(i)
    h = np.log(h[0:T])
    prod = np.log(prod)

    dhp, dtr = hp1(np.concatenate((y, i, c, k, h, prod),axis=1),1600)
    ss_mat[j,:] = np.std(dhp,axis=0,ddof=1)*100
    Corr = np.corrcoef(dhp,rowvar=False)
    cc_mat[j,:] = Corr[:,0]

std = np.mean(ss_mat,axis=0)
corr = np.mean(cc_mat,axis=0)

print('HANSEN: std(x)/std(y) corr(x,y) for y, i, c, k, h, prod:')
print(np.concatenate((np.array([[1.36, 4.24, 0.42, 0.36, 0.7, 0.68]]).T/1.36, \
                      np.array([[1, 0.99, 0.89, 0.06, 0.98, 0.98]]).T),axis=1))
print('std(x) std(x)/std(y) corr(x,y) for y, i, c, k, h, prod:')
print(np.concatenate((std[:,np.newaxis], (std/std[0])[:,np.newaxis], \
                      corr[:,np.newaxis]),axis=1))



HANSEN: std(x)/std(y) corr(x,y) for y, i, c, k, h, prod:
[[1.     1.    ]
 [3.1176 0.99  ]
 [0.3088 0.89  ]
 [0.2647 0.06  ]
 [0.5147 0.98  ]
 [0.5    0.98  ]]
std(x) std(x)/std(y) corr(x,y) for y, i, c, k, h, prod:
[[1.3417 1.     1.    ]
 [4.2282 3.1514 0.9903]
 [0.4164 0.3103 0.8954]
 [0.3591 0.2676 0.0693]
 [0.6899 0.5142 0.9824]
 [0.6766 0.5043 0.9818]]


In [10]:
#!jupyter nbconvert --to script hansenLQDP.ipynb